<a href="https://colab.research.google.com/github/sabumjung/Learning-Data-Mining-with-Python/blob/master/Mahalanobis_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mahalanobis Distance 계산
Mahalonobis 거리는 점과 분포 사이의 거리이다. 그리고 두 지점 사이의 거리를 정의하는 개념은 아니다. 사실상 다변량 변수에 대한 유클리드 거리 계산 방법이라고 할 수 있다.
Mahalanobis Distance를 계산하는 방법은 다음과 같다.
1. Columns를 상관관계가 없는 변수로 변환한다.
2. 해당 Columns를 정규화한다(분산이 1이 되도록).
3. 유틀리디안 거리를 계산한다.

마할라노비스 거리를 계산하는 공식은 다음과 같다.

마할라노비스 거리 공식

D^2 = (x-m)T * C-1 * (x-m)

여기서, 
 - D^2는 마할라노비스 거리의 제곱이다.
 - x는 관측치의 벡터이다(데이터 집합의 행). 
 - m은 독립 변수(각 열의 평균)의 평균값 벡터이다. 
 - C^(-1)은 독립 변수의 역 공분산 행렬이다. 

위의 공식을 어떻게 이해해야 할까요?

(x – m)^T. C^(-1) 항을 살표보도록 하지요.

(x – m)는 본질적으로 평균으로부터의 벡터 거리이다.
다음 이를 공분산 행렬(또는 공분산 행렬의 역행렬로 곱하기)로 나누기를 실행한다.

생각해 보면, 이는 본질적으로 정규 표준화(z = (x - mu)/다변량)에 해당하는 값이다.
즉, z = ((x 벡터) – (평균 벡터)) / (공분산 행렬) 이 된다.

그렇다면 공분산으로 나누면 어떤 효과가 있을까?

데이터 집합의 변수가 강하게 상관되어 있으면 공분산 값이 크다. 
큰 공분산으로 나누면 거리가 줄어드는 효과를 얻게된다.

마찬가지로 X가 상관 관계가 없으면 공분산 값이 크지 않게 된다.
따라서 Mahalanobis distance가 크게 줄어들지 않는다.

In [23]:
import pandas as pd
import scipy as sp
import numpy as np

In [24]:
filepath='https://raw.githubusercontent.com/selva86/datasets/master/diamonds.csv'

In [25]:
df=pd.read_csv(filepath).iloc[:,[0, 4,6]]

In [26]:
df.head()

,carat,depth,price
0,0.23,61.5,326
1,0.21,59.8,326
2,0.23,56.9,327
3,0.29,62.4,334
4,0.31,63.3,335


In [27]:
def mahalanobis(x=None, data=None, cov=None):
  x_minus_mu = x-np.mean(data)
  if not cov:
    cov=np.cov(data.values.T)
  inv_covmat =np.linalg.inv(cov)
  left_term = np.dot(x_minus_mu, inv_covmat)
  mahal = np.dot(left_term, x_minus_mu.T)
  return mahal.diagonal()

In [28]:
df_x = df[['carat', 'depth', 'price']].head(500)

In [29]:
df_x['mahala'] = mahalanobis(x = df_x, data = df[['carat', 'depth', 'price']])

In [30]:
df_x.head()

,carat,depth,price,mahala
0,0.23,61.5,326,1.709860
1,0.21,59.8,326,3.540097
2,0.23,56.9,327,12.715021
3,0.29,62.4,334,1.454469
4,0.31,63.3,335,2.347239


In [37]:
np.mean(df)

carat       0.797940
depth      61.749405
price    3932.799722
dtype: float64

In [38]:
np.var(df)

carat    2.246825e-01
depth    2.052366e+00
price    1.591533e+07
dtype: float64

# Mahalanobis Distance를 이용하여 Outlier를 detection하는 방법
자유도 n을 갖는 chi-square 분포를 따른다고 가정할 경우, 0.01 유의수준과 자유도 2에서는 하기와 같이 critical value를 계산할 수 있다.


In [39]:
# Critical values for two degrees of freedom
from scipy.stats import chi2

### 분석 결과 Mahanobis Distance가 9.21을 초과하는 경우 이상치로 검토할 수 있음을 의미한다.

In [40]:
chi2.ppf((1-0.01), df=2)

9.21034037197618

### 유의수준 0.01하에서 outlier에 해당 하는 값을 선정하면 다음과 같다.

In [41]:
df_x['p_value'] = 1 - chi2.cdf(df_x['mahala'], 2)
df_x.loc[df_x.p_value < 0.01].head(10)

,carat,depth,price,mahala,p_value
2,0.23,56.9,327,12.715021,0.001734
91,0.86,55.1,2757,23.909643,0.000006
97,0.96,66.3,2759,11.781773,0.002765
172,1.17,60.2,2774,9.279459,0.009660
204,0.98,67.9,2777,20.086616,0.000043
221,0.70,57.2,2782,10.405659,0.005501
227,0.84,55.1,2782,23.548379,0.000008
255,1.05,65.8,2789,11.237146,0.003630
284,1.00,58.2,2795,10.349019,0.005659
298,1.01,67.4,2797,17.716144,0.000142
